In [ ]:
#!/usr/bin/env python3

import rospy
import cv2
import numpy as np
from cv_bridge import CvBridge
from sensor_msgs.msg import Image
from geometry_msgs.msg import Twist

class SignalDetector:
    def __init__(self):
        rospy.init_node('signal_detector')
        self.bridge = CvBridge()

        # Cargar señal de referencia
        self.ref_img = cv2.imread('/home/user/catkin_ws/src/your_package/images/alto.jpg', cv2.IMREAD_GRAYSCALE)
        self.sift = cv2.SIFT_create()
        self.kp_ref, self.des_ref = self.sift.detectAndCompute(self.ref_img, None)
        self.bf = cv2.BFMatcher(cv2.NORM_L2)

        self.image_sub = rospy.Subscriber('/camera/rgb/image_raw', Image, self.image_callback)
        self.cmd_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)

        self.frame = None
        self.centered = False
        self.stop_distance_px = 230  # ancho en píxeles para detenerse

        rospy.loginfo("Detector iniciado.")
        rospy.spin()

    def image_callback(self, msg):
        self.frame = self.bridge.imgmsg_to_cv2(msg, 'bgr8')
        self.process_frame()

    def process_frame(self):
        if self.frame is None:
            return

        gray = cv2.cvtColor(self.frame, cv2.COLOR_BGR2GRAY)
        kp_frame, des_frame = self.sift.detectAndCompute(gray, None)

        if des_frame is None:
            return

        matches = self.bf.knnMatch(self.des_ref, des_frame, k=2)
        good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]

        twist = Twist()

        if len(good_matches) > 10:
            pts_ref = np.float32([self.kp_ref[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
            pts_frame = np.float32([kp_frame[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
            H, _ = cv2.findHomography(pts_ref, pts_frame, cv2.RANSAC, 5.0)

            if H is not None:
                h, w = self.ref_img.shape
                corners = np.float32([[0, 0], [0, h], [w, h], [w, 0]]).reshape(-1, 1, 2)
                pts_dst = cv2.perspectiveTransform(corners, H)

                w_obj = np.linalg.norm(pts_dst[0] - pts_dst[1])
                cx = np.mean([pt[0][0] for pt in pts_dst])
                center_frame = self.frame.shape[1] / 2

                if not self.centered:
                    if abs(cx - center_frame) < 30:
                        self.centered = True
                        twist.angular.z = 0.0
                        rospy.loginfo("Señal centrada.")
                    else:
                        twist.angular.z = 0.3 if cx > center_frame else -0.3
                else:
                    if w_obj < self.stop_distance_px:
                        twist.linear.x = 0.1
                    else:
                        twist.linear.x = 0.0
                        rospy.loginfo("Señal alcanzada. Tipo: ALTO")
        else:
            if not self.centered:
                twist.angular.z = 0.2

        self.cmd_pub.publish(twist)
